#### Generar nuevo DataFrame para el área de Gestión Documental, ya que tiene reglas de negocio y tiempos diferentes a los de TI

In [2]:
import os
from dotenv import load_dotenv
import pandas as pd
import numpy as np

load_dotenv()

%run conexionDB.ipynb

# Obtener la conexión
engine = obtener_conexion()

In [3]:
from datetime import datetime

# Fecha de inicio y salida a producción de la aplicación de soporte osTicket: 2 de febrero de 2019
fechaInicio = '2019-02-01'

# Fecha de fin: fecha actual (hoy)
fechaFin = datetime.now().strftime('%Y-%m-%d')

query_gestionDocumental = f"""
SELECT inf.number AS 'No. de Ticket', DATE_ADD(inf.created, INTERVAL -6 HOUR) AS 'Fecha de creación', asu.subject AS 'Asunto', cue.poster AS 'Usuario', cat.topic AS 'Tema de ayuda', DATE_ADD(age.timestamp, INTERVAL -6 HOUR) AS 'Fecha de cierre', DATE_ADD(inf.lastupdate, INTERVAL -6 HOUR) AS 'Ultima actualización'
FROM ost_ticket AS inf 
LEFT JOIN ost_thread_entry AS cue ON inf.created=cue.created 
LEFT JOIN ost_ticket__cdata AS asu ON inf.ticket_id=asu.ticket_id 
LEFT JOIN ost_help_topic AS cat ON inf.topic_id=cat.topic_id 
LEFT JOIN ost_thread_event AS age ON cue.thread_id=age.thread_id
LEFT JOIN ost_department AS dep ON inf.dept_id = dep.id
WHERE age.staff_id > 0 AND age.state = 'closed' AND inf.dept_id = 4 AND DATE_ADD(inf.created, INTERVAL -6 HOUR) BETWEEN '{fechaInicio}' AND '{fechaFin}'
GROUP BY inf.number
"""    
# Imprimir la consulta para verificar
print(query_gestionDocumental)
    
# Ejecutar la consulta y almacenar los resultados en la variable global
df_gestionDocumental = pd.read_sql(query_gestionDocumental, engine)
df_gestionDocumental


SELECT inf.number AS 'No. de Ticket', DATE_ADD(inf.created, INTERVAL -6 HOUR) AS 'Fecha de creación', asu.subject AS 'Asunto', cue.poster AS 'Usuario', cat.topic AS 'Tema de ayuda', DATE_ADD(age.timestamp, INTERVAL -6 HOUR) AS 'Fecha de cierre', DATE_ADD(inf.lastupdate, INTERVAL -6 HOUR) AS 'Ultima actualización'
FROM ost_ticket AS inf 
LEFT JOIN ost_thread_entry AS cue ON inf.created=cue.created 
LEFT JOIN ost_ticket__cdata AS asu ON inf.ticket_id=asu.ticket_id 
LEFT JOIN ost_help_topic AS cat ON inf.topic_id=cat.topic_id 
LEFT JOIN ost_thread_event AS age ON cue.thread_id=age.thread_id
LEFT JOIN ost_department AS dep ON inf.dept_id = dep.id
WHERE age.staff_id > 0 AND age.state = 'closed' AND inf.dept_id = 4 AND DATE_ADD(inf.created, INTERVAL -6 HOUR) BETWEEN '2019-02-01' AND '2025-01-11'
GROUP BY inf.number



,No. de Ticket,Fecha de creación,Asunto,Usuario,Tema de ayuda,Fecha de cierre,Ultima actualización
0,002371,2019-09-04 12:23:51,Apoyo Formato Legalizar viaticos,Pruebas TI,Apoyo Trámites,2019-09-04 12:27:12,2019-09-04 12:42:48
1,002394,2019-09-05 16:34:07,Douflow,ANIBAL FAJARDO BARRETO,Apoyo Trámites,2019-09-05 18:30:38,2019-09-05 18:30:38
2,002401,2019-09-06 10:50:27,461850 codigo de barras,JHOAN ARANGO SEGURA,Apoyo Trámites,2019-09-06 13:46:08,2019-09-06 13:46:08
3,002417,2019-09-09 11:28:46,Solicitud apoyo envío correo electrónico,JHOANA MONROY DELGADO,Apoyo Trámites,2019-09-09 11:43:50,2019-09-09 11:43:50
4,002472,2019-09-12 09:30:19,apoyo formato,JULIAN FELIPE MONTOYA RIVERA,Apoyo Trámites,2019-09-13 08:41:24,2019-09-13 08:41:24
...,...,...,...,...,...,...,...
761,040989,2025-01-09 09:37:03,Finalizacion de tramites,JOHAN ESNEIVER ROA MONTEALEGRE,Apoyo Trámites,2025-01-09 10:01:06,2025-01-09 10:01:06
762,041001,2025-01-09 14:52:50,verificacion en cargue de documentos,JORGE ALEJANDRO GUZMAN GARCIA,Reporte de Errores Docxflow,2025-01-10 10:05:44,2025-01-10 10:05:44
763,041017,2025-01-10 08:40:32,RENOVACION,INGRID TATIANA RIVERA HEREDIA,Reporte de Errores Docxflow,2025-01-10 10:14:01,2025-01-10 10:14:01
764,041026,2025-01-10 11:36:49,FINALIZACIÓN RAD CCI-IE25-5,DIEGO FERNANDO MONROY VERGARA,Apoyo Trámites,2025-01-10 11:46:18,2025-01-10 11:46:18
